This projekt is for vizualizing the data from the various sources for wifi (mainly with passwords)

Supported sources: 3WIFI database, pwnagotchi

This project is strongly inspirated (or put together from) 3WIFI and pwnmap projects

Licence: GPLv3

### Getting started

In [ ]:
#set up the environment
!python3 -m venv ./.venv
!pip install -r requirements.txt

In [ ]:
#Map start
!source ./.venv/bin/activate
!python3 src/main.py

User input (like config etc) is stored in map_app/sources/config/* For default values you have to run modules first (tun python3 app.py and go to tolls for example)

In [ ]:
!pip install requests tqdm
import os
import math
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Function to calculate tile coordinates from latitude and longitude
def deg_to_tile(lon, lat, zoom):
    x = int((lon + 180.0) / 360.0 * (2 ** zoom))
    y = int((1.0 - math.log(math.tan(math.radians(lat)) + 1 / math.cos(math.radians(lat))) / math.pi) / 2 * (2 ** zoom))
    return x, y

# Function to download a single tile
def download_tile(z, x, y, output_dir):
    url = f"https://tile.openstreetmap.org/{z}/{x}/{y}.png"
    output_path = os.path.join(output_dir, str(z), str(x))
    os.makedirs(output_path, exist_ok=True)
    tile_path = os.path.join(output_path, f"{y}.png")

    if os.path.exists(tile_path):
        return

    headers = {
        "User-Agent": "MyTileDownloader/1.0 (your_email@example.com)"  # Replace with your app name and email
    }

    try:
        response = requests.get(url, stream=True, timeout=10, headers=headers)
        if response.status_code == 200:
            with open(tile_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            #print(f"Downloaded: {url}")
        else:
            print(f"Failed: {url} (Status: {response.status_code})")
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")


# Function to download all tiles in a given bounding box
def download_tiles(min_zoom, max_zoom, min_lat, max_lat, min_lon, max_lon, output_dir):
    for z in range(min_zoom, max_zoom + 1):
        x_min, y_min = deg_to_tile(min_lon, max_lat, z)
        x_max, y_max = deg_to_tile(max_lon, min_lat, z)

        # Use ThreadPoolExecutor for parallel downloads
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = []
            for x in range(x_min, x_max + 1):
                for y in range(y_min, y_max + 1):
                    futures.append(executor.submit(download_tile, z, x, y, output_dir))
            for future in tqdm(futures, desc=f"Zoom level {z}", unit="tile"):
                future.result()  # Wait for all downloads to complete

if __name__ == "__main__":
    # Czechia Bounding Box
    czechia_min_lat = 48.55
    czechia_max_lat = 51.06
    czechia_min_lon = 12.09
    czechia_max_lon = 18.86

    # Albania Bounding Box
    albania_min_lat = 39.64
    albania_max_lat = 42.65
    albania_min_lon = 19.27
    albania_max_lon = 21.05

    output_dir = "data/tiles"  # Unified directory for all tiles

    # Download tiles for Czechia
    print("Downloading tiles for Czechia...")
    download_tiles(16,17, czechia_min_lat, czechia_max_lat, czechia_min_lon, czechia_max_lon, output_dir)

    # Download tiles for Albania
    #print("Downloading tiles for Albania...")
    #download_tiles(10, 15, albania_min_lat, albania_max_lat, albania_min_lon, albania_max_lon, output_dir)



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
